# Topic Analysis: BERT

Dataset used:


In [95]:
# Import libraries

from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gensim
import numpy as np
import sklearn
from sklearn.metrics import classification_report
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import matplotlib.pyplot as plt 
import seaborn as sn 
import nltk 
from nltk.stem import WordNetLemmatizer 
from sklearn import svm

In [96]:
from sklearn.datasets import fetch_20newsgroups

# load only a sub-selection of the categories (4 in our case)
categories = ['rec.sport.baseball', 'rec.sport.hockey', 'sci.med', 'sci.space'] 

# remove the headers, footers and quotes (to avoid overfitting)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=42)

label_mapping = {
    'rec.sport.baseball': 'sports',
    'rec.sport.hockey': 'sports',
    'sci.med': 'sci.med',
    'sci.space': 'sci.space'
}

# Map labels using the new names
labels_sp = [label_mapping[newsgroups_train.target_names[label]] for label in newsgroups_train.target]
print(labels_sp)






['sci.space', 'sports', 'sports', 'sci.med', 'sci.space', 'sci.med', 'sports', 'sports', 'sci.space', 'sci.space', 'sports', 'sports', 'sci.space', 'sci.med', 'sports', 'sports', 'sports', 'sci.med', 'sci.med', 'sports', 'sci.space', 'sci.space', 'sci.space', 'sci.med', 'sci.med', 'sports', 'sci.space', 'sports', 'sci.med', 'sci.med', 'sports', 'sci.med', 'sports', 'sci.med', 'sci.med', 'sports', 'sci.space', 'sci.med', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sci.space', 'sci.med', 'sports', 'sci.med', 'sports', 'sports', 'sports', 'sports', 'sci.med', 'sci.med', 'sci.med', 'sports', 'sports', 'sci.med', 'sports', 'sci.space', 'sci.med', 'sci.space', 'sports', 'sports', 'sports', 'sports', 'sci.space', 'sports', 'sports', 'sports', 'sports', 'sci.med', 'sports', 'sci.med', 'sci.med', 'sports', 'sports', 'sports', 'sports', 'sci.med', 'sports', 'sci.space', 'sports', 'sports', 'sci.space', 'sci.space', 'sports', 'sci.med', 'sports', 'sci.med', 'sports', 's

In [97]:
label_encoder = LabelEncoder()
new_labels = label_encoder.fit_transform(labels_sp)

In [98]:
train = pd.DataFrame({'text': newsgroups_train.data, 'labels': new_labels})

print(newsgroups_train.target_names)
print(train)


['rec.sport.baseball', 'rec.sport.hockey', 'sci.med', 'sci.space']
                                                   text  labels
0     \nA freeze dried Tootsie Roll (tm).  The actua...       1
1     \n\n\n Hmmm...what about walks and SB? Baerga ...       2
2     : >\n: >ATLANTIC DIVISION\n: >\t\n: >\tST JOHN...       2
3     \n\n    INTENSIVE JAPANESE AT THE UNIVERSITY O...       0
4     \n\n\n\n\nWhen I was at the Texas Star Party a...       1
...                                                 ...     ...
2379  KC(?) news was doing a report on that.  They s...       2
2380  There were a few people who responded to my re...       0
2381  There is a nice little tool in Lucid emacs. It...       1
2382  \nC'mon, Tommy Soderstrom is having a fine roo...       2
2383                                                          0

[2384 rows x 2 columns]


In [99]:
from sklearn.model_selection import train_test_split

train, dev = train_test_split(train, test_size=0.1, random_state=0, 
                               stratify=train[['labels']])

print(train['labels'])

628     0
581     2
922     0
58      0
941     0
       ..
932     1
815     1
2317    1
140     0
1041    0
Name: labels, Length: 2145, dtype: int64


In [100]:
# Model configuration # https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model 
model_args = ClassificationArgs()

model_args.overwrite_output_dir=True # overwrite existing saved models in the same directory
model_args.evaluate_during_training=True # to perform evaluation while training the model
# (eval data should be passed to the training method)

model_args.num_train_epochs=10 # number of epochs
model_args.train_batch_size=32 # batch size
model_args.learning_rate=4e-6 # learning rate
model_args.max_seq_length=256 # maximum sequence length
# Note! Increasing max_seq_len may provide better performance, but training time will increase. 
# For educational purposes, we set max_seq_len to 256.

# Early stopping to combat overfitting: https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model_args.use_early_stopping=True
model_args.early_stopping_delta=0.01 # "The improvement over best_eval_loss necessary to count as a better checkpoint"
model_args.early_stopping_metric='eval_loss'
model_args.early_stopping_metric_minimize=True
model_args.early_stopping_patience=2
model_args.evaluate_during_training_steps=32 # how often you want to run validation in terms of training steps (or batches)

In [101]:
model = ClassificationModel('bert', 'bert-base-cased', num_labels=4, args=model_args, use_cuda=False) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [102]:
_, history = model.train_model(train, eval_df=dev) 

  0%|          | 0/4 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true 

In [109]:
path = 'sentiment-topic-test.tsv'
file = pd.read_csv(path, sep="\t")


topics = file[['sentence', 'topic']].copy()

customlabels = {'sports': 2, 'book': 3, 'movie': 4}
topics['labels'] = topics['topic'].map(customlabels)

print(topics)



                                             sentence   topic  labels
0   The atmosphere at the stadium tonight was elec...  sports       2
1   The game was so intense I forgot to breathe at...  sports       2
2            It had me hooked from the first chapter.    book       3
3   It’s more of a slow burn than a page-turner, b...    book       3
4   It’s split into two timelines, which keeps it ...    book       3
5   I could watch this film a hundred times and st...   movie       4
6   Best thriller I’ve seen in ages. Had me on the...   movie       4
7   How do you concede three goals in ten minutes?...  sports       2
8   They rotated their squad for the cup game, whi...  sports       2
9   The trailer gave away most of the plot, but th...   movie       4
10  The protagonist was so whiny I wanted to throw...    book       3
11  The author’s writing style is so unique – poet...    book       3
12  I don't get how was it supposed to work withou...   movie       4
13  It's still 0-0 s

In [110]:
test = pd.DataFrame({'text': file['sentence'], 'labels': topics['labels']})
print(test)

                                                 text  labels
0   The atmosphere at the stadium tonight was elec...       2
1   The game was so intense I forgot to breathe at...       2
2            It had me hooked from the first chapter.       3
3   It’s more of a slow burn than a page-turner, b...       3
4   It’s split into two timelines, which keeps it ...       3
5   I could watch this film a hundred times and st...       4
6   Best thriller I’ve seen in ages. Had me on the...       4
7   How do you concede three goals in ten minutes?...       2
8   They rotated their squad for the cup game, whi...       2
9   The trailer gave away most of the plot, but th...       4
10  The protagonist was so whiny I wanted to throw...       3
11  The author’s writing style is so unique – poet...       3
12  I don't get how was it supposed to work withou...       4
13  It's still 0-0 so far, so way too early to tel...       2
14  I don't get the appeal at all, it's just a cou...       2
15  Did 

In [ ]:
predicted, probabilities = model.predict(test.text.to_list())
test['predicted'] = predicted

0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:03,  3.84s/it]
100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


In [112]:
test.head()

,text,labels,predicted
0,The atmosphere at the stadium tonight was elec...,2,2
1,The game was so intense I forgot to breathe at...,2,2
2,It had me hooked from the first chapter.,3,2
3,"It’s more of a slow burn than a page-turner, b...",3,1
4,"It’s split into two timelines, which keeps it ...",3,1


In [113]:
test

,text,labels,predicted
0,The atmosphere at the stadium tonight was elec...,2,2
1,The game was so intense I forgot to breathe at...,2,2
2,It had me hooked from the first chapter.,3,2
3,"It’s more of a slow burn than a page-turner, b...",3,1
4,"It’s split into two timelines, which keeps it ...",3,1
5,I could watch this film a hundred times and st...,4,1
6,Best thriller I’ve seen in ages. Had me on the...,4,2
7,How do you concede three goals in ten minutes?...,2,2
8,"They rotated their squad for the cup game, whi...",2,2
9,"The trailer gave away most of the plot, but th...",4,1


### Classification Report

In [114]:
print(classification_report(test['labels'], test['predicted']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.55      1.00      0.71         6
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         6

    accuracy                           0.33        18
   macro avg       0.11      0.20      0.14        18
weighted avg       0.18      0.33      0.24        18



/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anacond

### Result Analysis 

* changed 'rec.sport.baseball' to 'sports' and 'rec.sport.hockey' to 'sports' as label nr 2
* kept 'sci.med' (label 0) and 'sci.space' (label 1)
* made 'book' label 3 and 'movie' label 4 in order to prevent misidentification 
* classification report as expected: all labels on which the model didn't train scored 0.0 and the label 'sports' on which the model trained, had values for recall and precision. 